<a href="https://colab.research.google.com/github/rax125896343/rax/blob/main/raxlabmeeting_goodorbad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the [pytorch](https://pytorch.org) and [minicons](https://github.com/kanishkamisra/minicons) library

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 cpuonly -c pytorch
!pip install minicons -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully unins

Using pandas library to read the 'classifier_adj_1000.csv' file under '/content/drive/MyDrive/CLiMP_corpus/' directory

In [5]:
import pandas as pd
# 'Xuzhou/CLiMP_corpus/classifier_adj_1000.csv' depends on the folder path
df = pd.read_csv('/content/drive/My Drive/11/Emmanuele_Chersoni/CLiMP_corpus/classifier_adj_1000.csv', sep=',', encoding='utf-8')
print(f"sentences: {df['2']}")
print(f"labels: {df['3']}")

sentences: 0       赵诗涵正在探索一家待业的咖啡店
1       赵诗涵正在探索一件待业的咖啡店
2         王鑫快到达一个保密的杂货店
3         王鑫快到达一架保密的杂货店
4        张杰正在穿过一家整个艺术画廊
             ...       
1995     马可正在考虑一座丢失的投影机
1996       赵健杰看到了一把坏的菠菜
1997       赵健杰看到了一辆坏的菠菜
1998      詹静芳正在带一把快速的香菜
1999      詹静芳正在带一件快速的香菜
Name: 2, Length: 2000, dtype: object
labels: 0       1
1       0
2       1
3       0
4       1
       ..
1995    0
1996    1
1997    0
1998    1
1999    0
Name: 3, Length: 2000, dtype: int64


label==1 represents the classifier is correct, while label==0 means the classifier in the sentence is inappropriate.
Now, we will spilt the sentences into appropriate and inappropriate sets according to the labels

In [6]:
good_sentences = df[df['3']==1]
bad_sentences = df[df['3']==0]


import [scorer](https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md) from minicons to calculate the suprisals

In [7]:
from minicons import scorer
model = scorer.IncrementalLMScorer('uer/gpt2-chinese-cluecorpussmall', 'cpu')

In [ ]:
# from minicons import scorer
# model = scorer.MaskedLMScorer('bert-base-chinese', 'cpu')

Calculate the sentence log probability for good and bad sentences

In [9]:
# When vocabulary includes all tokens
#good_scores = []
#bad_scores = []
#for good, bad in zip(good_sentences['2'].tolist(), bad_sentences['2'].tolist()):
#  good_scores.extend(model.sequence_score(good, reduction = lambda x: x.mean(0)))
#  bad_scores.extend(model.sequence_score(bad, reduction = lambda x: x.mean(0)))
#print(good_scores)
#print(bad_scores)


# # select the first five pair of sentences when the codes work slowly
good_scores = []
bad_scores = []
for good, bad in zip(good_sentences['2'].tolist()[:3], bad_sentences['2'].tolist()[:3]):
  good_scores.extend(model.sequence_score(good, reduction = lambda x: x.mean(0).item()))
  bad_scores.extend(model.sequence_score(bad, reduction = lambda x: x.mean(0).item()))
print(good_scores)
print(bad_scores)

[-4.603067874908447, -5.168532371520996, -4.71627140045166]
[-5.015430927276611, -6.013216972351074, -5.343356609344482]


In [ ]:
# # When vocabulary does not include all tokens
# good_scores = []
# bad_scores = []
# for good, bad in zip(good_sentences['2'].tolist(), bad_sentences['2'].tolist()):
#   good_scores.extend(model.sequence_score(good, reduction = lambda x: x.mean(0).item(),PLL_metric='within_word_l2r'))
#   bad_scores.extend(model.sequence_score(bad, reduction = lambda x: x.mean(0).item(),PLL_metric='within_word_l2r'))
# print(good_scores)
# print(bad_scores)

Calculating the accuracy that grammatically correct sentences ('good') was assigned a greater log probability using the language model compared with grammatically incorrect sentences ('bad').




In [ ]:
import numpy as np
print(np.mean([g > b for g,b in zip(good_scores, bad_scores)]))

1.0
